In [1]:
using CPLEXW,Printf, MathProgBase, MathOptInterface, JuMP

function solve_callback(env, cbdata, wherefrom, cbhandle, useraction_p) :: Int32
        println("inside solve_callback")
        nodelp_p = [CPXLPptr(0)]
        CPXgetcallbacknodelp(env, cbdata, wherefrom, nodelp_p);
        CPXwriteprob(env, nodelp_p[1], string(input_file_path,"rootnodemodel.mps"), C_NULL);
        return 0
end 

c_solve_callback = @cfunction(solve_callback, Cint, (
        CPXENVptr,  # env
        Ptr{Cvoid}, # cbdata
        Cint,       # wherefrom
        Ptr{Cvoid}, # cbhandle
        Ptr{Cint},  # useraction_p
    ))

# input file path
input_file_path = "/Users/adminq/Downloads/case1888rte/"
inputfiles=readdir(input_file_path)
inputfiles = filter(x->occursin("mps",x), inputfiles)

for i in eachindex(inputfiles)[1:1]
    status_p = [Cint(0)] 
    env = CPXopenCPLEX(status_p)
    CPXsetintparam(env, CPX_PARAM_SCRIND, CPX_ON) # Print to the screen
    CPXsetdblparam(env, CPX_PARAM_NODELIM, 0)      # MIP node limit
    #CPXsetdblparam(env, CPX_PARAM_CUTSFACTOR, 0)  # Disable cutting planes
    #CPXsetintparam(env, CPX_PARAM_PREIND, 0)      # Disable preprocessing
    #CPXsetintparam(env, CPX_PARAM_HEURFREQ, -1)   # Disable primal heuristics
    lp = CPXcreateprob(env, status_p, "problem")
    CPXreadcopyprob(env, lp, string(input_file_path, inputfiles[i]), "mps")
    CPXsetsolvecallbackfunc(env, c_solve_callback, C_NULL)
    CPXmipopt(env, lp);
end



LoadError: ArgumentError: Package CPLEXW not found in current path:
- Run `import Pkg; Pkg.add("CPLEXW")` to install the CPLEXW package.


In [ ]:
m = JuMP.read_from_file(string(input_file_path,"rootnodemodel.mps"));
    write_to_file(m, string(input_file_path, inputfiles[1], ".lp");format = MOI.FileFormats.FORMAT_LP)
cons = all_constraints(m, GenericAffExpr{Float64,VariableRef}, MOI.LessThan{Float64});
#cons = [all_constraints(m, GenericAffExpr{Float64,VariableRef}, MOI.LessThan{Float64});all_constraints(m, GenericAffExpr{Float64,VariableRef}, MOI.EqualTo{Float64});all_constraints(m, GenericAffExpr{Float64,VariableRef}, MOI.GreaterThan{Float64})];

ib_cut_file = open("ib_cuts_training.txt", "a"); 
mir_cut_file = open("mir_cuts_training.txt", "a");
f_cut_file = open("f_cuts_training.txt", "a");
for conRef in cons
    if name(conRef)[begin] in ['i','m','f'] 
        c = constraint_object(conRef)
        strVarNames = ":"
        strCoeffs = ":"
        for term in c.func.terms
            strVarNames = string(strVarnames, ";", term.first)
            strCoeffs = string(strCoeffs, ":", term.second)
        end
    end
    
    if name(conRef)[begin] == 'i' && isdigit(name(conRef)[2]) # implied bound cuts
        println(name(conRef))
        c = constraint_object(conRef);
        varNames = JuMP.name.(keys(c.func.terms))  
        sort!(varNames)
        jointNames = join(varNames)
        csvLine= string(jointNames, ";",name(conRef),";", inputfiles[1], ";",conRef,"\n")        
        write(ib_cut_file, csvLine)   
    elseif name(conRef)[begin] == 'm' && isdigit(name(conRef)[2])
        c = constraint_object(conRef);
        varNames = JuMP.name.(keys(c.func.terms))  
        sort!(varNames)
        jointNames = join(varNames)
        csvLine= string(jointNames, ";",name(conRef),";", inputfiles[1], ";",conRef,"\n")        
        write(mir_cut_file, csvLine)   
    elseif name(conRef)[begin] == 'f' && isdigit(name(conRef)[2])
        c = constraint_object(conRef);
        varNames = JuMP.name.(keys(c.func.terms))  
        sort!(varNames)
        jointNames = join(varNames)
        csvLine= string(jointNames, ";",name(conRef),";", inputfiles[1], ";",conRef,"\n")        
        write(f_cut_file, csvLine)  
    else 
        c = constraint_object(conRef);
        varNames = JuMP.name.(keys(c.func.terms))  
        sort!(varNames)
        jointNames = join(varNames)
        csvLine= string(jointNames, ";",name(conRef),";", inputfiles[1], ";",conRef,"\n")       
        write(mir_cut_file, csvLine)                                   
    end

end    
            
close(ib_cut_file)     
close(mir_cut_file)
close(f_cut_file)